### Libraries

In [1]:
import pandas as pd
import numpy as np

### Small idea - find best top 10 based on few parameters
### ***PAPER for discussion each column / optmization can be weighted! for policy makers / urbanists 

- separate script
- e.g. top 100 - give points 
- add at least top 25! 


# FUNCTION

### INPUT DATA: combined_df 
### OUTPUT DATA: combined vehicles (based on chosen columns)

In [36]:
def compute_combined_optimization_scores(combined_df, columns=None, top_n=10):
    """
    Compute combined ranking scores from multiple optimization results.

    Parameters:
    - combined_df : DataFrame with optimization results (columns of ranked vehicle IDs).
    - columns     : Optional list of column names to use for scoring. If None, uses default set.
    - top_n       : Number of top vehicle IDs to return in the final output.

    Returns:
    - scores_df   : Full score table with optimization memberships.
    - top_n_df    : Top N vehicle IDs in a one-column DataFrame named 'combined_opt'.
    """
    if columns is None:
        columns = ['max_spatial', 'max_A_inhab', 'max_point_count', 'closest_vehicle_closest']

    df_sel = combined_df[columns].copy()
    n = len(df_sel)

    scores = {}
    for method in df_sel.columns:
        for rank, vid in enumerate(df_sel[method]):
            points = n - rank
            scores[vid] = scores.get(vid, 0) + points

    methods_map = {
        vid: [method for method in df_sel.columns if vid in df_sel[method].values]
        for vid in scores
    }

    scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['score'])
    scores_df['optimizations'] = scores_df.index.map(methods_map)
    scores_df = scores_df.sort_values('score', ascending=False)

    top_n_df = scores_df.head(top_n).reset_index()[['index']].rename(columns={'index': 'combined_opt'})
    return scores_df, top_n_df


In [42]:
# combined_df is list of all! startingwith that to evalueat
# only these cloest_

combined_df = pd.read_csv("data/combined_vehicles_1503.csv")

In [43]:
scores_combined_df, top_combined_final = compute_combined_optimization_scores(combined_df)

In [45]:
scores_combined_df

,score,optimizations
2268.0_GVB,58,"[max_spatial, max_A_inhab, closest_vehicle_clo..."
1921.0_GVB,50,"[max_spatial, max_A_inhab]"
309.0_GVB,35,"[max_spatial, max_A_inhab]"
2060.0_GVB,32,"[max_point_count, closest_vehicle_closest]"
310.0_GVB,31,"[max_spatial, max_point_count]"
...,...,...
2263.0_GVB,2,[max_A_inhab]
333.0_GVB,2,[max_point_count]
1104.0_GVB,1,[max_spatial]
2267.0_GVB,1,[max_A_inhab]


In [40]:
# export to csv
#top10_final.to_csv('data/optimized_combined_1503.csv', index=False)
